In [1]:
#https://blog.csdn.net/qq_42813323/article/details/88531940
import string

from nltk.tokenize import word_tokenize
from nltk import bigrams, FreqDist
from math import log

In [2]:
#tensorflow-gpu1.12.0
#numpy1.14.1
!conda list
#numpy1.14.1
# !python -m pip install --upgrade pip --trusted-host pypi.tuna.tsinghua.edu.cn
# !pip install -U -i https://pypi.tuna.tsinghua.edu.cn/simple numpy==1.19.5 --upgrade pip --trusted-host pypi.tuna.tsinghua.edu.cn

# packages in environment at D:\App\Anaconda3\envs\69t112:
#
# Name                    Version                   Build  Channel
absl-py                   1.3.0                    pypi_0    pypi
astor                     0.8.1                    pypi_0    pypi
blas                      1.0                         mkl    http://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
boto                      2.48.0                   py36_0    http://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
bz2file                   0.98                     py36_0    http://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
cached-property           1.5.2                    pypi_0    pypi
certifi                   2016.2.28                py36_0    http://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
colorama                  0.3.9                    py36_0    http://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
cudatoolkit               9.0                           1    http://mirrors.tuna.tsinghua.edu.

In [3]:
import nltk
nltk.find(".")

FileSystemPathPointer('D:\\App\\Anaconda3\\envs\\69t112\\lib\\nltk_data')

In [7]:
# 读取数据 小写 替换符号 分句
dataset = open("train_LM.txt", 'r+', encoding='utf-8').read().lower()\
                .replace(',',' ').replace('.',' ').replace('?',' ').replace('!',' ')\
                .replace(':',' ').replace(';',' ').replace('<',' ').replace('>',' ').replace('/',' ')\
                .split("__eou__")
testset = open("test_LM.txt", 'r+', encoding='utf-8').read().lower()\
                .replace(',',' ').replace('.',' ').replace('?',' ').replace('!',' ')\
                .replace(':',' ').replace(';',' ').replace('<',' ').replace('>',' ').replace('/',' ')\
                .split("__eou__")

In [6]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

uni-gram

In [8]:
#train
unigramsDist = FreqDist()  # uni-gram词频数字典
for i in dataset:
    sWordFreq = FreqDist(word_tokenize(i))  # 每一句的词频数字典
    for j in sWordFreq:
        if j in unigramsDist:
            unigramsDist[j] += sWordFreq[j]
        else:
            unigramsDist[j] = sWordFreq[j]

In [9]:
unigramsDist

FreqDist({'i': 53897, 'you': 48499, 'the': 38415, 'to': 32727, 'a': 27702, 'it': 24061, 'and': 16938, 'is': 16825, 'that': 15957, 'do': 14682, ...})

In [6]:
#test
# 加入未登录词
for i in testset:
    word = word_tokenize(i)  # 每一句的词频数字典
    for j in word:
        if j not in unigramsDist:
            unigramsDist[j] = 0

In [7]:
# 频数转化为频率  使用加一平滑法   unigramsDist.B()表示每个词都加一后的增加量
s = unigramsDist.N() + unigramsDist.B()
unigramsFreq = FreqDist()
for i in unigramsDist:
    unigramsFreq[i] = (unigramsDist[i] + 1) / s

In [8]:
ppt = []
for sentence in testset:
    logprob = 0
    wt = 0
    for word in word_tokenize(sentence):
        if word in unigramsFreq:
            logprob += log(unigramsFreq[word],2)
            wt += 1
    if wt > 0:
        ppt.append([sentence,pow(2,-(logprob/wt))])

In [9]:
temp = 0
for i in ppt:
    temp += i[1]
print("一元语法模型的困惑度:", temp/len(ppt))#885.5405291108561

一元语法模型的困惑度: 885.5405291108561


bi-gram

In [10]:
#train
w2gram = {}     # 可能存在的以w为开头的2-gram的种类数量
bigramsDist = FreqDist()
for sentence in dataset:
    sWordFreq = FreqDist(bigrams(word_tokenize(sentence)))
    for j in sWordFreq:
        if j in bigramsDist:
            bigramsDist[j] += sWordFreq[j]
        else:
            bigramsDist[j] = sWordFreq[j]
            if j[0] in w2gram:
                w2gram[j[0]] += 1
            else:
                w2gram[j[0]] = 1

In [11]:
#test
# 加入未登录词
# 由于将每种未出现的2-gram一一列举会生成vacab size * vocab size大小的bigramsDist，为节省时间和空间，此处只加入test中出现的2-gram
for sentence in testset:
    word = bigrams(word_tokenize(sentence))
    for j in word:
        if j not in bigramsDist:
            bigramsDist[j] = 0

            if j[0] in w2gram:
                w2gram[j[0]] += 1
            else:
                w2gram[j[0]] = 1

In [12]:
# 频数转化为频率  使用加一平滑法
history = {}    # 以w为历史的2-gram的数量和
for i in bigramsDist:
    if i[0] in history:
        history[i[0]] += bigramsDist[i]
    else:
        history[i[0]] = bigramsDist[i]
bigramsFreq = FreqDist()
for i in bigramsDist:
    bigramsFreq[i] = (bigramsDist[i] + 1) / (history[i[0]] + w2gram[i[0]])

In [13]:
ppt = []
for sentence in testset:
    logprob = 0
    wt = 0
    for word in bigrams(word_tokenize(sentence)):
        if word in bigramsFreq:
            logprob += log(bigramsFreq[word],2)
            wt += 1
    if wt > 0:
        ppt.append([sentence,pow(2,-(logprob/wt))])

In [14]:
temp = 0
for i in ppt:
    temp += i[1]
print("二元语法模型的困惑度:", temp/len(ppt))#68.33554824217941

二元语法模型的困惑度: 68.33554824217941
